In [1]:
# Code to load the necessary python modules
import ipywidgets as widgets
from IPython.display import display, Image, clear_output, HTML
import time
import random
random.seed(1)
from jupyter_ui_poll import ui_events

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

import os
import re

In [2]:
# function to display and use buttons
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):  
    start_wait = time.time()
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            ui_poll(n_proc)

            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False

            time.sleep(interval)

    return event_info

def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def button(top_area, main_area, bottom_area):
    ### set the button widget
    btn1 = widgets.Button(description='left')  
    btn2 = widgets.Button(description='right')  
    
    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)
    
    panel = widgets.HBox([btn1, btn2])
    
    top_area.append_display_data( HTML("<h3>Which side has more dots?</h3>") ) 
    bottom_area.append_display_data(panel)
    
    display(top_area)
    display(main_area)
    display(bottom_area)
    return top_area, main_area, bottom_area

In [3]:
# function to send the results to google form
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [4]:
def list_image(folder_path):
    # Create an empty list to store PNG files
    image_files = []

    # Get a list of all files in the specified folder
    all_files = os.listdir(folder_path)

    # Use list.append() to add only the PNG files to the image_files list
    for file in all_files:
        if file.endswith(".png") and file != "0-0.png": ## Exclude the blank image
            image_files.append(file)

    return image_files


folder_path = "ANS test pic"
images = list_image(folder_path)  # images is a list of all image filenames (.png)

In [5]:
# Function to match images with corresponding information
def match_image(filename):
    # Use regular expression to match and extract information from the filename
    match = re.match(r'(\d+)([by])-(\d+)([by])-?(\d*)\.png', filename)
            ## add comments
    
    # Check if the regular expression match was successful
    if match:
        # Extract information from the matched groups
        nL, cL, nR, cR = map(match.group, [1, 2, 3, 4])
        ## n: number of dots;  c: colour of dots
        ## L/R : left or right circle

        # Determine the answer based on the comparison of nL and nR
        if int(nL) > int(nR):
            answer = "left"
        else:
            answer = "right"
        
        # Return a tuple containing filename, extracted information, and answer
        return filename, nL, cL, nR, cR, answer

# Create a dictionary to store images and their information
pic_dict = {}

for i in range(len(images)):
   image_info = match_image(images[i])
   pic_dict[i] = image_info

## examples in pic_dict:
## {0: ('14y-12b.png', '14', 'y', '12', 'b', 'left'),
##  1: ('9b-10b-1.png', '9', 'b', '10', 'b', 'right'),...}


In [6]:
# function to create an empty dict to record the result in detail 
def init_results_dict():
    results_dict = {
        'filename': [],
        'nL': [],
        'nR': [],
        'ratio': [],
        'correct': [],
        'cL':[],
        'cR':[],
        'response_time':[]
    }
    return results_dict

# function to record the result that could be sent to the google form later
def record_result(results_dict, filename, nL, nR, ratio, 
                  correct, cL, cR, response_time):
    results_dict['filename'].append(filename)
    results_dict['nL'].append(nL)
    results_dict['nR'].append(nR)
    results_dict['ratio'].append(ratio)
    results_dict['correct'].append(correct)
    results_dict['cL'].append(cL)
    results_dict['cR'].append(cR)
    results_dict['response_time'].append(response_time)
    return 

In [7]:
def data_consent():
    data_consent_info = """DATA CONSENT INFORMATION:
    
    Please read:
    
    we wish to record your response data to an anonymised public data repository.
    
    Your data will be used for educational teaching purposes practising data analysis and visualisation.
    
    Please type "yes" in the box below if you consent to the upload."""
    print(data_consent_info)
    
    result = input("> ")
    if result == "yes":
        print("Thanks for your participation.")
        print("Please contact philip.lewis@ucl.ac.uk")
        print("If you have any questions or concerns")
        print("regarding the stored results.")
        clear_output(wait=False)
    else:
        # end code execution by raising an exception
        raise(Exception("User did not consent to continue test."))
    return

In [8]:
def record_inform():
    #### record the user_id
    id_instructions = """
    Please enter your anonymised ID.
    
    To generate an anonymous 4-letter unique user identifier please enter:
    - two letters based on the initials (first and last name) of a childhood friend
    - two letters based on the initials (first and last name) of a favourite actor / actress
    
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
    then your unique identifer would be CBTC
    
    """
    print(id_instructions)
    
    user_id = input("> ")
    print("User entered id:", user_id)
    clear_output(wait=False)

    #### record the age
    print("Please enter your age:")
    age = input("> ")
    clear_output(wait=False)

    #### record the gender
    print("Please enter your gender (m/f):")
    gender = input("> ")
    clear_output(wait=False)
    
    #### collect the basic information
    data_dict= {
        'user_id': user_id,
        'age': age,
        'gender': gender,
    }
    return data_dict

In [9]:
# function to run the ANS test
def run_ANS():
    ## before run the test
    ### create an empty dict to record the result in detail
    results_dict = init_results_dict()  
    ### add a data consent disclaimer 
    data_consent()
    
    ## run the test
    ### basic information of respondents
    print("Welcome to the ANS test! This is a test assessing your reaction time.")
    time.sleep(1)

    inform_dict = record_inform()
    
    ### give a brief introduction to the test
    print("The following image will be shown for 0.75 seconds and then removed from view.")
    time.sleep(1.5)

    print("Press the button to indicate which side has more dots once the image disappears.")
    time.sleep(2)
    
    ### set the screen
    top_area = widgets.Output(layout={"height":"60px"})
    main_area = widgets.Output(layout={"height":"250px"}) 
    bottom_area = widgets.Output(layout={"height":"80px"})
    
    ### set the button widget
    button(top_area, main_area, bottom_area)

    ### copy the list of keys to shuffle
    keys = list(pic_dict.keys())
    keys_copy = keys.copy()
    random.shuffle(keys_copy)

    ### record the initial total_score (0/64)
    total_score = 0  
    
    ### blank image
    pic_blank = Image("ANS test pic/0-0.png",width = 450)
    
    ### running the test with bottom & recording the data
    for i in keys_copy:
        ###pic_dict[i] shows (filename, nL, cL, nR, cR, answer)
        ###                     [0]    [1]  [2] [3] [4]  [5]
        with main_area: display(Image(f"ANS test pic/{pic_dict[i][0]}", width=450))
            
        #### record the time image appears
        start_time = time.time()
    
        #### image will disappear after 0.75s
        wait_for_event(timeout=0.75)
        with main_area: clear_output(wait=True)
        with main_area: display(pic_blank)

        #### wait for response within 3 sec
        result = wait_for_event(timeout=3)
        
        #### record response as NA (no response within 3 sec)
        if result['description'] == '':
           # with main_area: clear_output()
            time_taken = 0 ## record the reaction time as 0
            record_result(results_dict, f"pic{i}.png", int(pic_dict[i][1]),
                          int(pic_dict[i][3]), int(pic_dict[i][1])/int(pic_dict[i][3]), 
                          f"NA", pic_dict[i][2], pic_dict[i][4], time_taken)  
                   
        #### record response as correct "1"
        elif result['description'] ==  pic_dict[i][5]:
            total_score = total_score + 1 
            end_time = time.time() 
            time_taken = end_time - start_time ## record the reaction time
            record_result(results_dict, f"pic{i}.png", int(pic_dict[i][1]),
                          int(pic_dict[i][3]), int(pic_dict[i][1])/int(pic_dict[i][3]), 
                          1, pic_dict[i][2], pic_dict[i][4], time_taken)  
                         
        #### record response as incorrect "0"
        else:
            end_time = time.time()
            time_taken = end_time - start_time ## record the reaction time
            record_result(results_dict, f"pic{i}.png", int(pic_dict[i][1]),
                          int(pic_dict[i][3]), int(pic_dict[i][1])/int(pic_dict[i][3]), 
                          0, pic_dict[i][2], pic_dict[i][4], time_taken)  
        
        with main_area: clear_output() 
        # time.sleep(1) # short pause after each question has been answered
        
            
    ### clear the display and show the final total_score        
    with bottom_area: clear_output()
    
    with main_area: print(f"You scored {total_score:}. Well done! Thank you for your time :)")
                                 
    inform_dict['total_score'] =  total_score  
    ## this total_score should be an estimate of actual score ##
    ## score=0 when answer is incorrect or missed, score=1 when answer is correct ##
   
    ### change the dict into json
    results_df = pd.DataFrame(results_dict)
    inform_dict['results_json'] = results_df.to_json()
    
    ### send to google form
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdu6kmrYPpR_pi2htgQROxbnjnP___0_U-doAobSAkzAYddYA/viewform?usp=sf_link"
    send_to_google_form(inform_dict, form_url)
    return

In [10]:
## filename not correct (order of image name change as code change)

In [11]:
run_ANS()

The following image will be shown for 0.75 seconds and then removed from view.
Press the button to indicate which side has more dots once the image disappears.


Output(layout=Layout(height='60px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython…

Output(layout=Layout(height='250px'))

Output(layout=Layout(height='80px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chi…